# 模型忽略关键实体怎么办？注意力权重分配机制引导生成聚焦重点

## 场景痛点

大语言模型虽然强大，但在特定任务中仍可能出现“走神”现象。

例如，在生成摘要时遗漏核心人物名字，在问答系统中无法准确抽取关键日期，或在对话中忽略用户强调的重点。这些情况导致生成内容偏离主题，缺乏关键信息，甚至误导用户。

## 根本原因

模型在处理文本时，注意力机制未能对关键实体给予足够关注。Transformer 架构通过注意力权重对输入 token 进行加权聚合，从而决定输出内容。如果某些重要实体的注意力权重偏低，它们就很难在最终输出中体现出来。


### 方案一： 提示词

通过精心设计的 prompt，可以间接引导模型关注特定内容。例如，明确要求模型在回答中使用某些关键词，或以特定结构组织内容。这种方式简单有效，适用于大多数模型




In [ ]:
prompt = """
请用自然流畅的语言，深入探讨一下人工智能和大模型的未来发展趋势，并结合医疗、自动驾驶、智能客服等具体行业，分析它们的潜在应用和挑战。
请在你的回答中，尽可能自然地穿插以下词汇：大模型、人工智能、医疗、自动驾驶、智能客服。
"""

### 方案二： 自然语言处理
[命名实体识别](https://www.modelscope.cn/models/iic/nlp_seqgpt-560m)

借助命名实体识别技术，从输入中提取关键实体，并将其插入 prompt 或用于干预模型生成逻辑。该方法自动化程度高，能动态识别关键信息，但依赖外部模块，推理链更复杂。


### 方案三：修改 Attention 层，最终生成词汇的概率分布（logits）

更直接、有效的方式是干预模型输出层的 logits。logits 是模型对词汇表中每个词的“打分”，在它进入 softmax 之前修改，可以精确提升或降低特定词汇的生成概率。该方法不依赖 prompt，也不需要重新训练模型，适用于推理阶段实时干预。

In [1]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    LogitsProcessor,
    LogitsProcessorList,
)

# ================== 1. Load an Open-Source Model ==================
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"  # Change if needed

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
dtype = torch.float16 if device == "cuda" else torch.float32

print(f"Loading model: {MODEL_NAME} on {device}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=dtype,
    device_map="auto" if device == "cuda" else None,
).to(device)
model.eval()
print("Model loaded successfully!")

# ================== 2. Keywords and Token IDs ==================
keywords = [
    "large language models",
    "artificial intelligence",
    "healthcare",
    "autonomous driving",
    "intelligent customer service",
]

focus_token_ids = set()
for kw in keywords:
    ids = tokenizer.encode(kw, add_special_tokens=False)
    focus_token_ids.update(ids)

focus_token_ids = torch.tensor(
    sorted(focus_token_ids),
    device=device,
    dtype=torch.long,
)

print(f"Focused token IDs: {focus_token_ids.tolist()}")

# ================== 3. Logits Processor ==================
class KeywordBiasLogitsProcessor(LogitsProcessor):
    def __init__(self, token_ids: torch.Tensor, bias: float = 3.0):
        self.token_ids = token_ids
        self.bias = bias

    def __call__(self, input_ids, scores):
        scores[:, self.token_ids] += self.bias
        return scores

# ================== 4. Prompt Builder ==================
def build_chat_prompt(user_prompt: str) -> str:
    if hasattr(tokenizer, "apply_chat_template"):
        messages = [{"role": "user", "content": user_prompt}]
        return tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
    else:
        return f"User: {user_prompt}\nAssistant:"

# ================== 5. Text Generation ==================
@torch.no_grad()
def generate_text(prompt: str, use_bias: bool = False, bias: float = 3.0) -> str:
    text = build_chat_prompt(prompt)
    inputs = tokenizer(text, return_tensors="pt").to(device)

    logits_processor = None
    if use_bias:
        logits_processor = LogitsProcessorList([
            KeywordBiasLogitsProcessor(focus_token_ids, bias)
        ])

    output_ids = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.05,
        no_repeat_ngram_size=2,
        logits_processor=logits_processor,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )

    generated_ids = output_ids[0, inputs["input_ids"].shape[1]:]
    return tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

# ================== 6. Keyword Statistics ==================
def count_keywords(text: str, keywords):
    count = 0
    present = []
    for kw in keywords:
        c = text.lower().count(kw.lower())
        if c > 0:
            count += c
            present.append(kw)
    return count, present

# ================== 7. Test Prompt ==================
test_prompt = (
    "Please discuss the future development trends of artificial intelligence "
    "and large language models in a clear and natural manner, and analyze their "
    "potential applications and challenges across specific industries such as "
    "healthcare, autonomous driving, and intelligent customer service."
)

print("\n" + "=" * 20 + " Baseline Generation " + "=" * 20)
out1 = generate_text(test_prompt, use_bias=False)
print(out1)
c1, p1 = count_keywords(out1, keywords)
print(f"\n[Keyword Stats] count={c1}, present={p1}")

print("\n" + "=" * 20 + " Keyword-Biased Generation " + "=" * 20)
out2 = generate_text(test_prompt, use_bias=True, bias=3.0)
print(out2)
c2, p2 = count_keywords(out2, keywords)
print(f"\n[Keyword Stats] count={c2}, present={p2}")


/Users/linghuang/miniconda3/envs/llm_clean/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu
Loading model: Qwen/Qwen2.5-0.5B-Instruct on cpu...


`torch_dtype` is deprecated! Use `dtype` instead!


Model loaded successfully!
Focused token IDs: [396, 471, 2473, 2717, 4119, 4128, 6002, 9842, 10506, 11229, 12120, 16488, 16767, 20509, 29846]

==================== Baseline Generation ====================
As an AI language model, I am constantly evolving to understand the complex dynamics of today's world. Here is my perspective on the potential future developments in artificial Intelligence and Large Language Models (LLMs) across various industries:

1. Healthcare: The application of LLMs in healthcare can revolutionize the way doctors diagnose and treat diseases. By analyzing vast amounts of medical data, LLLs can provide insights into patient health patterns and predict potential health risks before they occur. This could lead to more personalized medicine, with treatments tailored to individual patients' genetic profiles and medical history. However, there are also ethical concerns about the use of AI in medical diagnostics, particularly when it comes to privacy and consent.

2. Au

### 通过干预 Logits 引导模型聚焦关键信息

对于像 Qwen 或 Llama 这样的先进自回归模型（Decoder-only Models），它们依赖自注意力机制来理解上下文。简而言之，模型在生成每个新词时，会回顾此前的所有文本，并从中提取相关信息。我们的目标是在这个“回顾”过程中施加影响，让模型更关注我们指定的关键内容。

## 高级干预策略

### 强力干预（The Hard Boost）

最直接的方式是在目标 token 的 logits 上添加一个固定的正向偏置。这种方式干预效果明显，但可能影响文本的自然性，导致关键词重复出现。可通过 `no_repeat_ngram_size` 等参数缓解这一问题。

### 温和引导（The Gentle Nudge）

更精细的做法是采用加权融合策略，将原始 logits 与关键词偏置进行线性融合：

```
new_logits = original_logits * (1 - α) + entity_bias * α
```

其中，α 是一个介于 0 和 1 之间的融合因子，用于控制干预强度。α 越小，干预越温和，生成内容越自然。

### 动态衰减

还可根据生成阶段动态调整偏置值。例如，在生成初期给予较强干预，随后逐步减弱，使模型在后期拥有更多自由发挥的空间，从而在聚焦关键信息与保持多样性之间取得平衡。

## 局限性

- **过度聚焦风险**：可能导致生成内容变得狭隘、重复，缺乏创造性。
- **计算开销**：虽然单次干预开销较小，但在复杂场景中频繁干预会略微增加推理延迟。

## 与其他技术的协同

强制聚焦并非孤立手段，它可与多种主流技术结合，实现更优效果：

- **提示词工程（Prompt Engineering）**：先用高质量 Prompt 指明方向，再通过干预确保关键细节不丢失。
- **RAG（检索增强生成）**：RAG 负责从外部知识库中检索关键信息，而干预技术则确保这些信息在最终输出中得以体现。
- **LoRA / QLoRA 微调**：通过微调让模型掌握特定领域知识，再在推理时用干预技术引导模型聚焦具体任务。

## 总结

通过钩子（Hook）机制干预模型的 Logits 层，是一种强大、可解释的干预方式。它能够引导模型在生成过程中聚焦关键实体，提升输出的准确性与相关性。结合提示词工程、RAG 和微调技术，可以进一步增强干预效果，使其在实际应用中更加稳定、自然地服务于特定任务需求。
